In [332]:
# -*- coding: utf-8 -*-
"""
math.ipynb
====================================
Basic script to create the mathematical model (no best practices)

@author:
     - j.rodriguez.villegas
"""

'\nmath.ipynb\n====================================\nBasic script to create the mathematical model (no best practices)\n\n@author:\n     - j.rodriguez.villegas\n'

In [333]:
# Coding the convex relaxed linear problem using Pyomo

#Import the libraries

import numpy as np
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po
import matplotlib.pyplot as plt

from pyomo.environ import *

In [334]:
# Declare the path to read the data
url = 'C:/Users/j.rodriguez.villegas/Documents/optimization/01_data/configuration_file.xlsx'

# Read the data
df = pd.read_excel(url, sheet_name = 'Historical_data_2')
df.head(15)

,Date,Asset,Open,High,Low,Close,Adj Close,Volume,ROI
0,2011-09-01,VOO,111.739998,112.800003,102.379997,103.519997,82.673691,4571000,-0.073564
1,2011-09-01,VXUS,46.110001,46.520000,39.250000,40.310001,27.907845,2370400,-0.125786
2,2011-09-01,BND,83.320000,84.580002,83.050003,83.739998,60.632889,27045300,0.005041
3,2011-09-01,VYM,42.650002,42.889999,39.459999,40.419998,27.762306,7184400,-0.052286
4,2011-09-01,VNQ,57.500000,57.959999,50.650002,50.869999,31.639059,54834000,-0.115304
5,2011-10-01,VOO,102.820000,118.300003,98.239998,114.639999,92.068909,8390200,0.114958
6,2011-10-01,VXUS,39.880001,47.200001,38.020000,44.439999,30.767185,4023800,0.114343
7,2011-10-01,BND,83.699997,84.050003,82.720001,83.620003,60.701000,35970000,-0.000956
8,2011-10-01,VYM,40.240002,44.950001,38.619999,43.680000,30.233118,9854700,0.085487
9,2011-10-01,VNQ,50.340000,59.119999,47.099998,58.139999,36.511829,60425700,0.154946


In [335]:
# Create a pivot table
pivot_table = df.pivot(index="Date", columns="Asset", values="ROI")

# Calculate the covariance between asset returns (ROI)
covariance_matrix = pivot_table.cov()

# Get the list of unique assets
assets = df["Asset"].unique()

# Calculate and print the covariances for all combinations of products
for i in range(len(assets)):
    for j in range(i, len(assets)):
        asset1 = assets[i]
        asset2 = assets[j]
        covariance_prod = covariance_matrix.loc[asset1, asset2]
        print(f"Covariance ({asset1}, {asset2}): {covariance_prod}")

Covariance (VOO, VOO): 0.0018746454341649205
Covariance (VOO, VXUS): 0.0016558184367195786
Covariance (VOO, BND): 0.00012617885417384226
Covariance (VOO, VYM): 0.001601850364433954
Covariance (VOO, VNQ): 0.0016459148651173819
Covariance (VXUS, VXUS): 0.001971801713188773
Covariance (VXUS, BND): 0.00016608044485428905
Covariance (VXUS, VYM): 0.001457910942347234
Covariance (VXUS, VNQ): 0.001537435888074811
Covariance (BND, BND): 0.00013496252192067074
Covariance (BND, VYM): 6.821630302185274e-05
Covariance (BND, VNQ): 0.00026996698460159005
Covariance (VYM, VYM): 0.0015896249495842463
Covariance (VYM, VNQ): 0.0014476249000060903
Covariance (VNQ, VNQ): 0.002587724045259413


In [336]:
# Create a pivot table
pivot_table = df.pivot(index="Date", columns="Asset", values="ROI")

# Calculate the correlation between asset returns (ROI)
correlation_matrix = pivot_table.corr()

# Get the list of unique assets
assets = df["Asset"].unique()

# Calculate and print the covariances for all combinations of products
for i in range(len(assets)):
    for j in range(i, len(assets)):
        asset1 = assets[i]
        asset2 = assets[j]
        correlation_prod = correlation_matrix.loc[asset1, asset2]
        print(f"Correlation ({asset1}, {asset2}): {correlation_prod}")

Correlation (VOO, VOO): 1.0
Correlation (VOO, VXUS): 0.8612347104176759
Correlation (VOO, BND): 0.2508536459929909
Correlation (VOO, VYM): 0.9279296243597943
Correlation (VOO, VNQ): 0.747289392603855
Correlation (VXUS, VXUS): 1.0
Correlation (VXUS, BND): 0.3219439640692268
Correlation (VXUS, VYM): 0.8234780747654172
Correlation (VXUS, VNQ): 0.6806226757496999
Correlation (BND, BND): 1.0
Correlation (BND, VYM): 0.1472767701881342
Correlation (BND, VNQ): 0.4568199717231082
Correlation (VYM, VYM): 1.0
Correlation (VYM, VNQ): 0.7137559287510205
Correlation (VNQ, VNQ): 1.0


In [337]:
# Calculate the expected return for each asset
# Create a new DataFrame
assets_df = df.groupby("Asset")["ROI"].mean().reset_index()
assets_df.rename(columns={"ROI": "Mean (average) ROI"}, inplace=True)

assets_df.head(10)

,Asset,Mean (average) ROI
0,BND,0.001051
1,VNQ,0.003556
2,VOO,0.008735
3,VXUS,0.000080
4,VYM,0.006128


In [338]:
# Optimization model (Convex linear relaxation)

assets_data = assets_df

# Sets
model = pe.ConcreteModel('markowitz')
model.assets = pe.Set(initialize = assets_data['Asset'].drop_duplicates())

assets_data = assets_data.set_index('Asset')

In [339]:
print("The number of elements in the set {model.assets} is: ", len(model.assets))

The number of elements in the set {model.assets} is:  5


In [340]:
# Parameters
model.return_level = pe.Param(initialize = 0.001)
model.risk_level = pe.Param(initialize = 0.0001)
model.expected_return = pe.Param(model.assets, initialize = assets_data['Mean (average) ROI'].to_dict())

covariance_dict = {}
for i in range(len(assets)):
    for j in range(i, len(assets)):
        asset1 = assets[i]
        asset2 = assets[j]
        covariance_prod = covariance_matrix.loc[asset1, asset2]
        
        if asset1 not in covariance_dict:
            covariance_dict[asset1] = {}
        covariance_dict[asset1][asset2] = covariance_prod

model.covariance = pe.Param(model.assets, model.assets, initialize=0.0, mutable=True)

# Assign the covariance values to the parameter (asset1, asset2)
for asset1, covariance_i in covariance_dict.items():
    for asset2, covariance_prod in covariance_i.items():
        model.covariance[asset1, asset2] = covariance_prod

In [341]:
# Display the values of model.covariance parameter
for asset1 in model.assets:
    for asset2 in model.assets:
        covariance_param = model.covariance[asset1, asset2].value
        print(f"Covariance ({asset1}, {asset2}): {covariance_param:.8f}")

Covariance (BND, BND): 0.00013496
Covariance (BND, VNQ): 0.00026997
Covariance (BND, VOO): 0.00000000
Covariance (BND, VXUS): 0.00000000
Covariance (BND, VYM): 0.00006822
Covariance (VNQ, BND): 0.00000000
Covariance (VNQ, VNQ): 0.00258772
Covariance (VNQ, VOO): 0.00000000
Covariance (VNQ, VXUS): 0.00000000
Covariance (VNQ, VYM): 0.00000000
Covariance (VOO, BND): 0.00012618
Covariance (VOO, VNQ): 0.00164591
Covariance (VOO, VOO): 0.00187465
Covariance (VOO, VXUS): 0.00165582
Covariance (VOO, VYM): 0.00160185
Covariance (VXUS, BND): 0.00016608
Covariance (VXUS, VNQ): 0.00153744
Covariance (VXUS, VOO): 0.00000000
Covariance (VXUS, VXUS): 0.00197180
Covariance (VXUS, VYM): 0.00145791
Covariance (VYM, BND): 0.00000000
Covariance (VYM, VNQ): 0.00144762
Covariance (VYM, VOO): 0.00000000
Covariance (VYM, VXUS): 0.00000000
Covariance (VYM, VYM): 0.00158962


In [342]:
# Display the values of the model.expected_return parameter
for asset in model.assets:
    expected_return = model.expected_return[asset]
    print(f"Expected return ({asset}): {expected_return:.8f}")

Expected return (BND): 0.00105145
Expected return (VNQ): 0.00355644
Expected return (VOO): 0.00873477
Expected return (VXUS): 0.00007966
Expected return (VYM): 0.00612764


In [343]:
# Decision variables
model.w = pe.Var(model.assets, domain = pe.NonNegativeReals, bounds = (0,1))
model.u = pe.Var(model.assets, model.assets, domain = pe.NonNegativeReals)

In [344]:
# Objective functions

# Maximization 
def calculate_total_return(model):
    '''
    This function calculates the total expected return on investment.

    Parameters
    ----------
    model : Pyomo ConcreteModel
        The optimization model.

    Return
    ------------
    double
        Total expected return.
    '''
    total_return = sum(model.expected_return[i] * model.w[i] for i in model.assets)
    return total_return

# Minimization
def calculate_total_risk(model):
    '''
    This function calculates the total risk of the portfolio.

    Parameters
    ----------
    model : Pyomo ConcreteModel
        The optimization model.

    Return
    ------------
    double
        Total risk.
    '''
    total_risk = sum(model.u[i,j] * model.covariance[i,j] for i in model.assets for j in model.assets)
    return total_risk

In [345]:
# Constraints
def c_1_weights(model):
    '''
    Ensures the total weight sums up to one.

    Parameters
    ----------
    model : Pyomo ConcreteModel
        The optimization model.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    weight_left = sum(model.w[i] for i in model.assets)
    weight_right = 1
    weight = (weight_left == weight_right)
    return weight

def c_2_return(model):
    '''
    Ensures the desired level of return of the portfolio is accomplished

    Parameters
    ----------
    model : Pyomo ConcreteModel
        The optimization model.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    expected_return_left = sum(model.expected_return[i] * model.w[i] for i in model.assets)
    expected_return_right = model.return_level
    expected_return = (expected_return_left >= expected_return_right)
    return expected_return

def c_3_1_linearized_risk(model):
    '''
    Linearizes the original MPT quadratic constraint by introducing an auxiliary variable.

    Parameters
    ----------

    model : Pyomo ConcreteModel
        The optimization model.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    auxiliary_1_left = sum(model.u[i,j] * model.covariance[i,j] for i in model.assets for j in model.assets)
    auxiliary_1_right = model.risk_level
    auxiliary_1 = (auxiliary_1_left <= auxiliary_1_right)
    return auxiliary_1

def c_3_2_linearized_risk(model, i, j):
    '''
    Assigns bounds to the auxiliary variable for the linear convex relaxation (McCormick Envelopes).

    Parameters
    ----------

    model : Pyomo ConcreteMode
        The optimization model.
    i : string
        The assets.
    j : string.
        The assets.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    lowBound = 0
    auxiliary_2_left = model.u[i,j]
    auxiliary_2_right = lowBound * model.w[j] + model.w[i] * lowBound  -  lowBound * lowBound
    auxiliary_2 = (auxiliary_2_left >= auxiliary_2_right)
    return auxiliary_2

def c_3_3_linearized_risk(model, i, j):
    '''
    Assigns bounds to the auxiliary variable for the linear convex relaxation (McCormick Envelopes).

    Parameters
    ----------

    model : Pyomo ConcreteMode
        The optimization model.
    i : string
        The assets.
    j : string.
        The assets.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    upperBound = 1
    auxiliary_3_left = model.u[i,j]
    auxiliary_3_right = upperBound * model.w[j] + model.w[i] * upperBound  -  upperBound * upperBound
    auxiliary_3 = (auxiliary_3_left >= auxiliary_3_right)
    return auxiliary_3

def c_3_4_linearized_risk(model, i, j):
    '''
    Assigns bounds to the auxiliary variable for the linear convex relaxation (McCormick Envelopes).

    Parameters
    ----------

    model : Pyomo ConcreteMode
        The optimization model.
    i : string
        The assets.
    j : string.
        The assets.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    lowBound = 0
    upperBound = 1
    auxiliary_4_left = model.u[i,j]
    auxiliary_4_right = upperBound * model.w[j] + model.w[i] * lowBound  -  upperBound * lowBound
    auxiliary_4 = (auxiliary_4_left <= auxiliary_4_right)
    return auxiliary_4

def c_3_5_linearized_risk(model, i, j):
    '''
    Assigns bounds to the auxiliary variable for the linear convex relaxation (McCormick Envelopes).

    Parameters
    ----------

    model : Pyomo ConcreteMode
        The optimization model.
    i : string
        The assets.
    j : string.
        The assets.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    lowBound = 0
    upperBound = 1
    auxiliary_5_left = model.u[i,j]
    auxiliary_5_right = model.w[i] * upperBound  + lowBound * model.w[j]  -  lowBound * upperBound
    auxiliary_5 = (auxiliary_5_left <= auxiliary_5_right)
    return auxiliary_5

def c_4_non_negative(model, i):
    '''
    Non negativity in the decision variable.

    Parameters
    ----------

    model : Pyomo ConcreteMode
        The optimization model.
    i : string
        The assets.
    j : string.
        The assets.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    non_negative_left = model.w[i]
    non_negatiave_right = 0
    non_negative = (non_negative_left >= non_negatiave_right)
    return non_negative


In [346]:
# Solve the optimization model

from pyomo.environ import SolverFactory, SolverStatus, TerminationCondition

model.c_1_weights = pe.Constraint(rule = c_1_weights)
model.c_2_return = pe.Constraint(rule = c_2_return)
model.c_3_1_linearized_risk = pe.Constraint(rule = c_3_1_linearized_risk)
model.c_3_2_linearized_risk = pe.Constraint(model.assets, model.assets, rule = c_3_2_linearized_risk)
model.c_3_3_linearized_risk = pe.Constraint(model.assets, model.assets, rule = c_3_3_linearized_risk)
model.c_3_4_linearized_risk = pe.Constraint(model.assets, model.assets, rule = c_3_4_linearized_risk)

model.obj_calculate_total_return = pe.Objective(sense = pe.maximize, rule = calculate_total_return)

solver = pe.SolverFactory('glpk')

# Set the time limit in seconds
time_limit = 600  # 600 seconds

# GLPK configurations
solver.options['tmlim'] = time_limit

# Solve the linear convex problem
result = solver.solve(model, tee = True, report_timing = True)

# Check the solver's termination condition
termination_condition = result.solver.termination_condition
solver_status = result.solver.status

# Print the results or handle them accordingly
if solver_status == SolverStatus.ok and termination_condition == TerminationCondition.optimal:
    print("Optimal solution found.")
    pe.value(model.obj_calculate_total_return)
elif solver_status == SolverStatus.ok and termination_condition in [TerminationCondition.maxTimeLimit]:
    print("Time limit reached. No optimal solution found within the specified time.")
else:
    print("No optimal solution found, model is infeasible or unbounded...")
    print("Solver terminated with condition:", solver_status)

        0.00 seconds required to write file
        0.01 seconds required for presolve
GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 600 --write C:\Users\JRODRI~1.VIL\AppData\Local\Temp\tmp80u3sl2z.glpk.raw
 --wglp C:\Users\JRODRI~1.VIL\AppData\Local\Temp\tmphkyk634u.glpk.glp --cpxlp
 C:\Users\JRODRI~1.VIL\AppData\Local\Temp\tmp8al10x47.pyomo.lp
Reading problem data from 'C:\Users\JRODRI~1.VIL\AppData\Local\Temp\tmp8al10x47.pyomo.lp'...
79 rows, 31 columns, 171 non-zeros
451 lines were read
Writing problem data to 'C:\Users\JRODRI~1.VIL\AppData\Local\Temp\tmphkyk634u.glpk.glp'...
373 lines were written
GLPK Simplex Optimizer 5.0
79 rows, 31 columns, 171 non-zeros
Preprocessing...
53 rows, 30 columns, 145 non-zeros
Scaling...
 A: min|aij| =  6.822e-05  max|aij| =  2.000e+00  ratio =  2.932e+04
GM: min|aij| =  3.090e-01  max|aij| =  3.236e+00  ratio =  1.047e+01
EQ: min|aij| =  9.550e-02  max|aij| =  1.000e+00  ratio =  1.047e+01
Constructing initial

In [347]:
print("Expected return: ", pe.value(model.obj_calculate_total_return))

Expected return:  0.007500741537351056


In [348]:
model.w.pprint()

w : Size=5, Index=assets
    Key  : Lower : Value             : Upper : Fixed : Stale : Domain
     BND :     0 :               0.0 :     1 : False : False : NonNegativeReals
     VNQ :     0 :               0.0 :     1 : False : False : NonNegativeReals
     VOO :     0 : 0.526671710334532 :     1 : False : False : NonNegativeReals
    VXUS :     0 :               0.0 :     1 : False : False : NonNegativeReals
     VYM :     0 : 0.473328289665468 :     1 : False : False : NonNegativeReals
